# Here, I build out the Langevin integrator capacities of my system. 

In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

In [ ]:
N= 10 # number of particles
p = np.empty((N,3),dtype=float) # momentum vector
q= p.copy() # position vector
M = np.empty((N),dtype=float) # mass vector
dt = .001 # integrator timestep
num_steps = 100 # total simulation time
T = 10 # temperature
gamma = .5 # friction
kB = 1.e-1 # what should this be??


We want to apply B A O A B. In the README.md file attached, we describe these terms in more detail. The important thing to note here is that A depends on p and is applied to q, while B depends on q and is applied to p. The beauty of BAOAB is that it allows us to freeze q wrt p and p wrt q for the A and B steps by examining variable-specific flows. Hence, integrating these operations over dt is equivalent to integrating a constant. In particular, we have $$\int \frac{dq}{dt}dt = \int \frac{M}{p}dt \implies \Delta q = \Delta t\frac{M}{p} $$ with a similar result for $B$. However, the $O$ operation is applied to $p$ and depends on $p$. We have $$\int \frac{dp}{dt} = \int -\gamma p + \xi(t) \sqrt{2\gamma k_B Tm} \ dt.$$ Letting $\sigma = \sqrt{2\gamma k_B Tm}$ and $\xi(t)dt = W_t$, a vector of $N$ independent Wiener processes. Then we use a technique common in the soft sciences (physics, eg.) and multiply by $e^{\gamma t}$ to get $$\begin{aligned}  d(e^{\gamma t}p) = \gamma e^{\gamma t}p dt + e^{\gamma t}dp = \gamma e^{\gamma t}pdt + e^{\gamma t}(-\gamma p dt + \sigma d W_t)  = \sigma e^{\gamma t}dW_t \end{aligned}.$$ We can then integrate as physicists, getting $$   \int d(e^{\gamma t}p) = \sigma \int e^{\gamma t} dW_t  \implies e^{\gamma (t + dt)}p(t+dt) - e^{\gamma t}p(t) = \sigma \int _t ^{t + dt} e^{\gamma t} dW_s$$ where the RHS is a bizarre integral over our stochastic function. Then simple algebra gives $$  p(t + dt) = e^{-\gamma dt}p(t) + \sigma \int_t^{t + dt} e^{\gamma (s-t-dt)}dW_s   $$

In [ ]:
def apply_A(p,q,dt):
    # the B operator doesn't touch p, but it adjusts q (position) via Newton
    q += p/M * dt # careful! do i need to recast M to adjust all slots of p, or will it figure that out?
    return(p,q)

def apply_B(p,q,dt):
    # the A operator applies the newtonian force kick 
    F = -1*getNablaU(q)
    p += F*dt
    return(p,q)

O_coeff= np.sqrt(2*gamma*kB*T)
def apply_O(p,q,dt,O_coeff = O_coeff):
    # the O operator adds thermal randomness to our system
    # it only adjusts p

    



    xi = np.random.randn(np.shape(p)) # how do i do this? want to make an array of randomness
    p += xi * O_coeff*dt

    return(p,q)

## Run the Simulation

In [ ]:
def run(p0 = p,q0=q,N=N,M=M,dt=dt,num_steps=num_steps,T=T,gamma=gamma,kB=kB):
    # perform BAOAB

    p=p0
    q=q0

    for step in range(num_steps):
        p,q = apply_B(p,q,dt/2)
        p,q = apply_A(p,q,dt/2)
        p,q = apply_O(p,q,dt)
        p,q = apply_A(p,q,dt/2)
        p,q = apply_B(p,q,dt/2)
    


    return p,q